# Time Frequency analysis in ECoG

This week's lab will explore the frequency representation of ECoG. We'll calculate the full time-frequency representation of an ECoG signal, and then focus on a particular frequency band of interest.

We'll now be going back into the world of ECoG, to see what differences we find. We'll use the same "consonant/dissonant" dataset seen in previous weeks. Perhaps by investigating the time-frequency representation of the signal we will uncover things that we couldn't see before...

In [ ]:
import mne
import numpy as np
import datascience as ds
import neurods as nds
import matplotlib.pyplot as plt
%matplotlib notebook

* Load in the raw data from the consonant/dissonant task
* Also load the image of the brain and layout of ECoG channels
* Finally, load in the event timing information for this task.

In [ ]:
path_ecog = '../../data/ecog/chords_task/'
data_path = path_ecog + 'ecog_resamp-raw.fif'
event_path = path_ecog + 'meta_time.csv'

### STUDENT ANSWER
raw = mne.io.Raw(data_path, preload=True)
events = ds.Table.read_table(event_path, index_col=0)
im = plt.imread(path_ecog + 'brain.png')
lt = mne.channels.read_layout('brain', path_ecog)

* Convert the events information into an MNE events object. Make sure to include all event types in the events object.
* Also create an events dictionary that maps event types onto event ID numbers.

In [ ]:
### STUDENT ANSWER
ev_ixs = events['start'] * raw.info['sfreq']
ev_ixs = ev_ixs.astype(int)
ev_types = np.where(events['type'] == 'consonant', 1, 2)
ev_mne = np.vstack([ev_ixs, np.zeros_like(ev_ixs), ev_types]).T
ev_dict = dict(consonant=1, dissonant=2)

* Using this events array and our events dictionary, convert the `Raw` data to an `Epochs` object
* Plot the global field power for each event type.

In [ ]:
### STUDENT ANSWER
epochs = mne.Epochs(raw, ev_mne, event_id=ev_dict,
                    tmin=-.5, tmax=.5, preload=True)

In [ ]:
### STUDENT ANSWER
# First, plot the global power of each stim type (for all channels)
f, ax = plt.subplots()
for dtype in ['dissonant', 'consonant']:
    i_data = epochs[dtype]._data.mean(axis=0) ** 2
    i_data = i_data.mean(axis=0)
    ax.plot(epochs.times, i_data)

This is the plot we generated last week. It looks like the differences between conditions is relatively small, so let's see if something else comes out of a time-frequency analysis.

* Convert the `Epochs` object into a time-frequency representation.
  * Use frequencies from 4 to 140 spaced 2Hz apart
  * Accomplish this with the neurods `tfr_morlet` function.
  * Print the shape of the output for each condition. What does each axis represent?

In [ ]:
### STUDENT ANSWER
freqs = np.arange(4, 140, 2)
tfrs = {'consonant': [], 'dissonant': []}
for dtype in ['consonant', 'dissonant']:
    tfrs[dtype] = nds.tfr.tfr_morlet(epochs[dtype]._data, epochs.info['sfreq'],
                                     freqs, average=True, use_fft=True)

* For each condition:
  * Convert the output of `tfr_morlet` into an `AverageTFR` object. This will let you plot it. Use the docstring from `AverageTFR` to figure out how to create an object from data.
  * Plot a topographic map of the TFR for each electrode. Use the `plot_tfr_topo` function in neurods.
  * Plot an image of the brain in the background. For this, use the `add_background_image` function in `mne.viz`.
  * In the call to `plot_topo`, use `mode='zscore', vmin=-5, vmax=5, cmap=plt.cm.coolwarm`

In [ ]:
### STUDENT ANSWER
for condition in ['consonant', 'dissonant']:
    avtfr = mne.time_frequency.AverageTFR(epochs.info, tfrs[condition],
                                          epochs.times, freqs,
                                          tfrs[condition].shape[0])
    fig = nds.viz.plot_tfr_topo(avtfr, im, lt)

* Finally, plot the difference (dissonant - consonant) for each TFR. Convert this into an MNE `AverageTFR` object.
* Plot the result on the brain as well.

In [ ]:
### STUDENT ANSWER
diff = tfrs['dissonant'] - tfrs['consonant']

avtfr = mne.time_frequency.AverageTFR(epochs.info, diff,
                                      epochs.times, freqs,
                                      tfrs['consonant'].shape[0])
_ = nds.viz.plot_tfr_topo(avtfr, im, lt)

* Do you see any differences between the two groups?
* How would you test for a difference between the two?

# Inspecting high-frequency activity
It seems like there are two "blobs" of high-frequency activity in the bove plots. Moreover, these blobs are more localized to specific electrodes than the ERPs of raw activity we saw before. Let's take a look at this activity.

* Extract the first Epoch from the consonant data. Insert it into a new variable.
* Using a bank of morlet wavelets, filter the epoch with 5 equally-spaced frequencies from 70-140Hz.
* Take the "real" component of the result (using `np.real`), and average across frequencies.

In [ ]:
### STUDENT ANSWER
freqs = np.linspace(70, 140, 5)
one_epoch = epochs._data[30]
tfr = mne.time_frequency.cwt_morlet(one_epoch, epochs.info['sfreq'], freqs)
tfr = np.real(tfr).mean(1)

* Plot the result in a single plot

In [ ]:
### STUDENT ANSWER
f, ax = plt.subplots()
_ = ax.plot(epochs.times, tfr.T)

* What do you notice about the edges of the plot?
* Try the same process with a few other epochs, see if the edges look consistent.

The peaks that you see at the corners are called "edge artifacts". In order to get around them, we need to remove timepoints near the edges of our data (e.g., at the beginning and end). As such, it is generally better to extract the frequency amplitudes from the **`Raw`** data, and then convert it to epochs.

* Using the `extract_amplitude` function in `neurods`, extract 5 linearly-spaced amplitudes from 70-150 Hz (define them manually or try using `np.linspace`). This function does the same thing that you did above, but it is more efficient with memory use.
* Turn this `Raw` object into an `Epochs` object using the same event times that we used above.

In [ ]:
### STUDENT ANSWER
raw_amp = nds.tfr.extract_amplitude(raw, freqs)
ep_amp = mne.Epochs(raw_amp, ev_mne, event_id=ev_dict,
                    tmin=-.5, tmax=.5, preload=True)

* Create an average response for each event type 
* Plot this average response (with the `.plot` method)
* For make a list of the top 3 active channels that you determine by visual inspection (you can see the channel names by clicking on each trace)

In [ ]:
### STUDENT ANSWER
av_c = ep_amp['consonant'].average()
av_d = ep_amp['dissonant'].average()

_ = av_c.plot()
_ = av_d.plot()

In [ ]:
### STUDENT ANSWER
active_chs = ['ch_57', 'ch_49', 'ch_50']

* Now, plot each one on the brain using the ECoG layout we loaded above. Remember we can do this with the `plot_topo` function, and don't forget to include an image of the brain as well (which we've also already loaded).

In [ ]:
### STUDENT ANSWER
_ = av_c.plot_topo(lt, fig_background=im)
_ = av_d.plot_topo(lt, fig_background=im)

* Finally, plot the difference (dissonant - consonant) between the two on the brain

In [ ]:
### STUDENT ANSWER
av_diff = av_d - av_c
_ = av_diff.plot_topo(lt, fig_background=im)

* Does it look like there is a difference between the two?
* Is it more or less localized than the signals that we've looked at so far?
* Why or why not?

In [ ]:
### STUDENT ANSWER